# Import

In [34]:
### import docs pickle from drive

from google.colab import drive
drive.mount('/content/drive/')
! cp -r --verbose '/content/drive/MyDrive/ESEO/I3/S9/Langage_naturel/TP_issou/model' .
! cp -r --verbose '/content/drive/MyDrive/ESEO/I3/S9/Langage_naturel/TP_issou/vect' .

MODEL_FILE = './model'
VECT_FILE = './vect'

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
'/content/drive/MyDrive/ESEO/I3/S9/Langage_naturel/TP_issou/model' -> './model'
'/content/drive/MyDrive/ESEO/I3/S9/Langage_naturel/TP_issou/vect' -> './vect'


In [35]:
import pickle as pk

In [36]:
# Open docs pickle

model = pk.load(open(MODEL_FILE, 'rb'))
vect = pk.load(open(VECT_FILE, 'rb'))
print(vect)

TfidfVectorizer(max_df=0.36, min_df=0.05,
                stop_words=frozenset({'a', 'about', 'above', 'across', 'after',
                                      'afterwards', 'again', 'against', 'all',
                                      'almost', 'alone', 'along', 'already',
                                      'also', 'although', 'always', 'am',
                                      'among', 'amongst', 'amoungst', 'amount',
                                      'an', 'and', 'another', 'any', 'anyhow',
                                      'anyone', 'anything', 'anyway',
                                      'anywhere', ...}))


In [37]:
TEXT = ["Textblob is kinda bad to use. I don't like sushi but the ambiance was ok"]

# Polarité

In [38]:
from textblob import TextBlob

testimonial1 = TextBlob(str(TEXT))
print(str(TEXT))
print(testimonial1.sentiment.polarity)

["Textblob is kinda bad to use. I don't like sushi but the ambiance was ok"]
-0.09999999999999992


In [39]:
def negative(text):
  negative = False
  testimonial1 = TextBlob(text)
  if testimonial1.sentiment.polarity < 0:
    negative = True
  return negative 

In [40]:
negative("wasn't very pretty today, so I will not come a day like today")

False

# Prétraitement

In [41]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
!pip3 install contractions
import contractions

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [42]:
tokenizer = RegexpTokenizer(r'\w+')

def tokenize_text(text):
    text_processed = " ".join(tokenizer.tokenize(text))
    return text_processed

In [43]:
tokenize_text("wasn't very pretty today, so I will not come a day like today")

'wasn t very pretty today so I will not come a day like today'

In [44]:
import en_core_web_sm
nlp = en_core_web_sm.load(disable=['parser', 'tagger', 'ner'])

lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    
    tokens_tagged = nltk.pos_tag(nltk.word_tokenize(text))
    lemmatized_text_list = list()
    
    for word, tag in tokens_tagged:
        if tag.startswith('J'):
            lemmatized_text_list.append(lemmatizer.lemmatize(word,'a')) # Lemmatise adjectives. Not doing anything since we remove all adjective
        elif tag.startswith('V'):
            lemmatized_text_list.append(lemmatizer.lemmatize(word,'v')) # Lemmatise verbs
        elif tag.startswith('N'):
            lemmatized_text_list.append(lemmatizer.lemmatize(word,'n')) # Lemmatise nouns
        elif tag.startswith('R'):
            lemmatized_text_list.append(lemmatizer.lemmatize(word,'r')) # Lemmatise adverbs
        else:
            lemmatized_text_list.append(lemmatizer.lemmatize(word)) # If no tags has been found, perform a non specific lemmatisation
    
    return " ".join(lemmatized_text_list)

/usr/local/lib/python3.7/dist-packages/spacy/language.py:1899: UserWarning: [W123] Argument disable with value ['parser', 'tagger', 'ner'] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.
  config_value=config["nlp"][key],


In [45]:
%%time
lemmatize_text("The brown fox is quick and he is jumping over the lazy dog")

CPU times: user 2.12 s, sys: 114 ms, total: 2.24 s
Wall time: 2.45 s


'The brown fox be quick and he be jump over the lazy dog'

In [46]:
def normalize_text(text):
    return " ".join([word.lower() for word in text.split()])

In [47]:
normalize_text("The brown fox is quick and he is jumping over the lazy dog")

'the brown fox is quick and he is jumping over the lazy dog'

In [48]:
def contraction_text(text):
    return contractions.fix(text)

In [49]:
contraction_text("The brown fox isn't quick and he wouldn't jumping over the lazy dog")

'The brown fox is not quick and he would not jumping over the lazy dog'

In [50]:
negative_words = ['not', 'no', 'never', 'nor', 'hardly', 'barely']
negative_prefix = "NOT_"

In [51]:
def get_negative_token(text):
    tokens = text.split()
    negative_idx = [i+1 for i in range(len(tokens)-1) if tokens[i] in negative_words]
    for idx in negative_idx:
        if idx < len(tokens):
            tokens[idx]= negative_prefix + tokens[idx]
    
    tokens = [token for i,token in enumerate(tokens) if i+1 not in negative_idx]
    
    return " ".join(tokens)

In [52]:
get_negative_token("I will never do that again, because I will not be foolish again, there is no risk, never")

'I will NOT_do that again, because I will NOT_be foolish again, there is NOT_risk, never'

In [53]:
from spacy.lang.en.stop_words import STOP_WORDS

def remove_stopwords(text):
    english_stopwords = stopwords.words("english") + list(STOP_WORDS) + ["tell", "restaurant"]
    
    return " ".join([word for word in text.split() if word not in english_stopwords])

In [54]:
remove_stopwords("I will never do that again, because I will not be foolish again, there is no risk, never")

'I again, I foolish again, risk,'

Préprocess final

In [55]:
def preprocess_text(text):
    text = tokenize_text(text)
    text = lemmatize_text(text)
    text = normalize_text(text)
    text = contraction_text(text)
    text = get_negative_token(text)
    text = remove_stopwords(text)
    return text

In [56]:
text_test = preprocess_text("I will never do that again, because I will not be foolish again, there is no risk, never")
print(text_test)

NOT_do NOT_be foolish NOT_risk


# vectorization

In [57]:
import numpy as np

my_vertorizer = vect.transform(TEXT)
my_model = model.transform(my_vertorizer)

print(np.argsort(my_model))
print(my_model[0][0], my_model[0][-1])

[[ 1  5  6  9 13 14  2 10 12  3  7  8 11  4  0]]
0.01652309329991388 0.0


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but NMF was fitted with feature names
  "X does not have valid feature names, but"


In [58]:
topic_nuns = ['atmosphere_sound', 'chicken_menu', 'bad_service', 'pizza_menu', 'delivery', 'long_wait', 'drinks', 'wrong_marketing', 'dirty', 'rude_staff', 'burger_menu', 'over_priced', 'not_tasty', 'not_accessible', 'seasoning']
nb_topic = 2

In [59]:
print(np.argsort(my_model)) #list des topic les plus probable dans l'ordre décroissant
print(my_model[0][0], my_model[0][-1]) #comparaison du dernier(0) et du premier(-1)
for i in np.argsort(my_model): #inversement
  print(i)

[[ 1  5  6  9 13 14  2 10 12  3  7  8 11  4  0]]
0.01652309329991388 0.0
[ 1  5  6  9 13 14  2 10 12  3  7  8 11  4  0]


In [64]:
model = pk.load(open(MODEL_FILE, 'rb'))
vect = pk.load(open(VECT_FILE, 'rb'))

def topic_search(text, nb_topic, model=model, vectorizer=vect):
  my_vertorizer = vect.transform(text)
  my_model = model.transform(my_vertorizer)
  topics = []
  for i in np.argsort(my_model):
    for j in range(nb_topic):
      topics.append(topic_nuns[i[-j-1]])
  return topics

In [65]:
print(topic_search(TEXT, 3))

['atmosphere_sound', 'delivery', 'over_priced']


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but NMF was fitted with feature names
  "X does not have valid feature names, but"


# Transformation en fichier .py

In [66]:
%%writefile /content/drive/MyDrive/ESEO/I3/S9/Langage_naturel/TP_issou/appp.py

MODEL_FILE = './model'
VECT_FILE = './vect'


#polarite


from textblob import TextBlob

def negative(text):
  negative = False
  testimonial1 = TextBlob(text)
  if testimonial1.sentiment.polarity < 0:
    negative = True
  return negative 


#pretraitement


import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import contractions

tokenizer = RegexpTokenizer(r'\w+')

def tokenize_text(text):
    text_processed = " ".join(tokenizer.tokenize(text))
    return text_processed

import en_core_web_sm
nlp = en_core_web_sm.load(disable=['parser', 'tagger', 'ner'])

lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    tokens_tagged = nltk.pos_tag(nltk.word_tokenize(text))
    lemmatized_text_list = list()
    for word, tag in tokens_tagged:
        if tag.startswith('J'):
            lemmatized_text_list.append(lemmatizer.lemmatize(word,'a')) # Lemmatise adjectives. Not doing anything since we remove all adjective
        elif tag.startswith('V'):
            lemmatized_text_list.append(lemmatizer.lemmatize(word,'v')) # Lemmatise verbs
        elif tag.startswith('N'):
            lemmatized_text_list.append(lemmatizer.lemmatize(word,'n')) # Lemmatise nouns
        elif tag.startswith('R'):
            lemmatized_text_list.append(lemmatizer.lemmatize(word,'r')) # Lemmatise adverbs
        else:
            lemmatized_text_list.append(lemmatizer.lemmatize(word)) # If no tags has been found, perform a non specific lemmatisation
    return " ".join(lemmatized_text_list)

def normalize_text(text):
    return " ".join([word.lower() for word in text.split()])

def contraction_text(text):
    return contractions.fix(text)

negative_words = ['not', 'no', 'never', 'nor', 'hardly', 'barely']
negative_prefix = "NOT_"

def get_negative_token(text):
    tokens = text.split()
    negative_idx = [i+1 for i in range(len(tokens)-1) if tokens[i] in negative_words]
    for idx in negative_idx:
        if idx < len(tokens):
            tokens[idx]= negative_prefix + tokens[idx]
    
    tokens = [token for i,token in enumerate(tokens) if i+1 not in negative_idx]
    
    return " ".join(tokens)

from spacy.lang.en.stop_words import STOP_WORDS

def remove_stopwords(text):
    english_stopwords = stopwords.words("english") + list(STOP_WORDS) + ["tell", "restaurant"]
    
    return " ".join([word for word in text.split() if word not in english_stopwords])

def preprocess_text(text):
    text = tokenize_text(text)
    text = lemmatize_text(text)
    text = normalize_text(text)
    text = contraction_text(text)
    text = get_negative_token(text)
    text = remove_stopwords(text)
    return text


#vectorization


import pickle as pk
import numpy as np

topic_nuns = ['atmosphere_sound', 'chicken_menu', 'bad_service', 'pizza_menu', 'delivery', 'long_wait', 'drinks', 'wrong_marketing', 'dirty', 'rude_staff', 'burger_menu', 'over_priced', 'not_tasty', 'not_accessible', 'seasoning']

  model = pk.load(open(MODEL_FILE, 'rb'))
  vect = pk.load(open(VECT_FILE, 'rb'))

def topic_search(text, nb_topic, model=model, vectorizer=vect):
  my_vertorizer = vect.transform(text)
  my_model = model.transform(my_vertorizer)
  topics = []
  for i in np.argsort(my_model):
    for j in range(nb_topic):
      topics.append(topic_nuns[i[-j-1]])
  return topics



Overwriting /content/drive/MyDrive/ESEO/I3/S9/Langage_naturel/TP_issou/appp.py
